# CDC State and County Policy Measures

* [COVID-19 State and County Policy Orders](https://healthdata.gov/dataset/covid-19-state-and-county-policy-orders)

In [ ]:
import pandas as pd
import numpy as np
import pycountry
import json
from datetime import datetime
from functools import reduce
import requests
from io import StringIO
import re

In [ ]:
# papermill parameters
output_folder = "../output/"

In [ ]:
# Get today's path

y, m, d = datetime.now().year, datetime.now().month, datetime.now().day

url = f"https://healthdata.gov/sites/default/files/state_policy_updates_{y}{m:02d}{d:02d}_0720.csv"


In [ ]:
states = [subdivision.code.replace("US-", "") for subdivision in pycountry.subdivisions if subdivision.code.startswith("US-")]

In [ ]:
def grab_and_augment_table(url: str) -> pd.DataFrame:
    try:
        df = pd.read_csv(url)
        df["ISO3166_1"] = "US"
        df["ISO3166_2"] = df["state_id"]
        df["DATE"] = f"{y}-{m}-{d}"
        return df[df["ISO3166_2"].isin(states)]
    except Exception as e:
        pass

In [ ]:
reported_policy_updates = grab_and_augment_table(url)

In [ ]:
reported_policy_updates.to_csv(output_folder + "CDC_POLICY_MEASURES.csv", index=False, columns=reported_policy_updates.columns)